# Deep Learning Scorer simple LSTM
Going to create multiple files so I can run them separately with less concern

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from keras import backend as k
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Conv2D, MaxPooling2D, Dense, Conv1D, Dropout
from keras.layers import MaxPooling1D, Flatten, Embedding, LSTM
from keras.models import Sequential
import spacy
import nltk
import en_core_web_md

Using TensorFlow backend.


In [2]:
ign_df = pd.read_pickle('ign_data/ign_data_pass_three.pkl')

ign_df = ign_df.loc[ign_df['review_score']!='null']
ign_df.reset_index(drop=True)

print(ign_df.shape)
ign_df.head()

(11072, 19)


,links,rough_review_body,cleaned_review_body,review_summary_rough,review_score,tags,categories,modified_date,published_date,author_name,tag_list,categories_list,review_title,review_body_corpus,review_score_float,number_character_review,number_words_review,polarity,subjectivity
1,https://www.ign.com/articles/2011/07/19/warham...,"\u003csection class=\""article-page\""\u003e\u00...",'Warhammer 40K: Kill Team might be unfortun...,"""And \""also-ran\"" hangs all over Warhammer 40K...",6.5,":[""review"",""blogroll"",""event-essentials"",""hot""...",":[""ign"",""ps3"",""xbox-360"",""xbox-live""],",2011-10-18T05:59:32+0000,2011-07-19T19:47:00+0000,"""Arthur Gies""","[review, blogroll, event-essentials, hot, lega...","[ign, ps3, xbox-360, xbox-live, ]","""Warhammer 40K: Kill Team Review""",warhammer 40k kill team might be unfortunat...,6.5,3705,642,0.062923,0.433371
2,https://www.ign.com/articles/2011/07/20/call-o...,"\u003csection class=\""article-page\""\u003eThe ...",The Call of Juarez franchise always intereste...,"""Call of Juarez: The Cartel is a poor change o...",4.5,":[""review"",""blogroll"",""call-of-juarez-the-cart...",":[""ign"",""pc"",""ps3"",""xbox-360""],",2011-10-18T06:02:02+0000,2011-07-20T00:14:00+0000,"""Anthony Gallegos""","[review, blogroll, call-of-juarez-the-cartel, ...","[ign, pc, ps3, xbox-360, ]","""Call of Juarez: The Cartel Review""",the call of juarez franchise always intereste...,4.5,3770,648,-0.016471,0.457898
3,https://www.ign.com/articles/2011/07/20/captai...,"\u003csection class=\""article-page\""\u003eThe ...",The majority of movie games are just awful. S...,"""Captain America: Super Soldier is a mediocre ...",5.5,":[""review"",""blogroll"",""legacy"",""wii"",""games"",""...",":[""ign"",""wii""],",2011-10-18T06:02:20+0000,2011-07-20T00:43:00+0000,"""Audrey Drake""","[review, blogroll, legacy, wii, games, captain...","[ign, wii, ]","""Captain America: Super Soldier Wii Review""",the majority of movie games are just awful st...,5.5,2675,474,-0.003842,0.544458
4,https://www.ign.com/articles/2011/07/20/quiz-c...,"\u003csection class=\""article-page\""\u003e\u00...","'Quiz Climber, from Buzz! developer 'Rele...","""Ultimately, Quiz Climber is just a little too...",6.5,":[""review"",""blogroll"",""games"",""legacy"",""wirele...",":[""ign"",""wireless""],",2011-10-18T06:07:06+0000,2011-07-20T21:44:00+0000,"""Justin Davis""","[review, blogroll, games, legacy, wireless, qu...","[ign, wireless, ]","""Quiz Climber Review""",quiz climber from buzz developer relentle...,6.5,1854,349,0.091160,0.600205
5,https://www.ign.com/articles/2011/07/20/limbo-...,"\u003csection class=\""article-page\""\u003eVide...",Video games are an art form made up of visual...,"""Limbo is an incredible achievement. Very few ...",9,":[""review"",""games"",""legacy"",""limbo"",""platforme...",":[""pc"",""ps3""],",2011-10-18T06:07:21+0000,2011-07-20T22:29:00+0000,"""Daemon Hatfield""","[review, games, legacy, limbo, platformer, pla...","[pc, ps3, ]","""Limbo Review""",video games are an art form made up of visual...,9.0,2938,527,0.096551,0.525587


In [3]:
# min/max_df are document frequency
vectorizer = CountVectorizer(binary=True, stop_words=stopwords.words('english'), 
                             lowercase=True, min_df=3, max_df=0.9, max_features=5000)

test_size = 0.2

ign_data = ign_df['cleaned_review_body'].values
ign_scores = ign_df['review_score_float'].values 

data_train, data_validation, target_train, target_validation = train_test_split(ign_data, 
                                                                                ign_scores, 
                                                                                test_size=test_size)

In [4]:
# starting training for ohe
score_space = np.linspace(start=0.0,stop=10.0, num=101)
temp_values = [[round(x, 2)] for x in score_space]
# temp_values = [[str(round(x, 2))] for x in score_space]
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(temp_values)
# print(enc.transform([[0.0], [0.3], [1.5]]).toarray())
ign_score_format = [[round(x, 2)] for x in ign_scores]
# ign_score_format = [[x] for x in ign_scores]
ign_targets = enc.transform(ign_score_format)

In [5]:
# creating one hot encoding
train_target_format = [[round(x, 2)] for x in target_train]
valid_target_format = [[round(x, 2)] for x in target_validation]

target_train_ohe = enc.transform(train_target_format)
target_valid_ohe = enc.transform(valid_target_format)

print(target_train_ohe.shape)
print(target_valid_ohe.shape)

(8857, 101)
(2215, 101)


In [6]:
vectorizer = CountVectorizer(binary=True, stop_words=stopwords.words('english'), 
                             lowercase=True, min_df=3, max_df=0.9, max_features=5000)
data_train_cv = vectorizer.fit_transform(data_train)
data_valid_cv = vectorizer.transform(data_validation)

In [7]:
word2idx = {word: idx for idx, word in enumerate(vectorizer.get_feature_names())}
tokenize = vectorizer.build_tokenizer()
preprocess = vectorizer.build_preprocessor()

# print(len(vectorizer.get_feature_names()))

def to_sequence(tokenizer, preprocessor, index, text):
    
    words = tokenizer(preprocessor(text))
    indexes = [index[word] for word in words if word in index]
    
    return indexes

print(to_sequence(tokenize, preprocess, word2idx, 'This is an important test!'))

[2208, 4430]


In [8]:
data_train_sequences = [to_sequence(tokenize, preprocess, word2idx, x) for x in data_train]
data_valid_sequences = [to_sequence(tokenize, preprocess, word2idx, x) for x in data_validation]

# padding sequences
MAX_SEQ_LENGTH = len(max(data_train_sequences, key=len))
if len(max(data_valid_sequences, key=len)) > MAX_SEQ_LENGTH:
    MAX_SEQ_LENGTH = len(max(data_valid_sequences, key=len))
    
N_FEATURES = len(vectorizer.get_feature_names())

data_train_sequences = pad_sequences(data_train_sequences, maxlen=MAX_SEQ_LENGTH, value=N_FEATURES)
data_valid_sequences = pad_sequences(data_valid_sequences, maxlen=MAX_SEQ_LENGTH, value=N_FEATURES)

print(data_train_sequences[0])
# print(vectorizer.get_feature_names())
# print(vectorizer.build_tokenizer())

[5000 5000 5000 ... 2443 4030  332]


In [11]:
def simple_lstm_model():
    
    model = Sequential()
    
    model.add(Embedding(len(vectorizer.get_feature_names()) +1, 64, 
                        input_length=MAX_SEQ_LENGTH))
    
    model.add(LSTM(64))
    model.add(Dense(units=101, activation='softmax'))
    
    return model

simple_lstm = simple_lstm_model()
simple_lstm.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(simple_lstm.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 2703, 64)          320064    
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_2 (Dense)              (None, 101)               6565      
Total params: 359,653
Trainable params: 359,653
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
simple_lstm.fit(data_train_sequences[:-100], target_train_ohe[:-100], 
                   epochs=100, batch_size=512, verbose=1, 
                   validation_data=(data_train_sequences[-100:], target_train_ohe[-100:]) )

Train on 8757 samples, validate on 100 samples
Epoch 1/100
8757/8757 [==============================] - ETA: 40s - loss: 4.6152 - accuracy: 0.007 - ETA: 36s - loss: 4.6142 - accuracy: 0.011 - ETA: 32s - loss: 4.6133 - accuracy: 0.020 - ETA: 29s - loss: 4.6121 - accuracy: 0.024 - ETA: 27s - loss: 4.6110 - accuracy: 0.033 - ETA: 25s - loss: 4.6096 - accuracy: 0.040 - ETA: 22s - loss: 4.6081 - accuracy: 0.044 - ETA: 20s - loss: 4.6064 - accuracy: 0.049 - ETA: 18s - loss: 4.6046 - accuracy: 0.052 - ETA: 15s - loss: 4.6024 - accuracy: 0.055 - ETA: 13s - loss: 4.5997 - accuracy: 0.058 - ETA: 11s - loss: 4.5965 - accuracy: 0.060 - ETA: 9s - loss: 4.5926 - accuracy: 0.064 - ETA: 6s - loss: 4.5876 - accuracy: 0.06 - ETA: 4s - loss: 4.5810 - accuracy: 0.06 - ETA: 2s - loss: 4.5716 - accuracy: 0.06 - ETA: 0s - loss: 4.5555 - accuracy: 0.06 - 41s 5ms/step - loss: 4.5533 - accuracy: 0.0659 - val_loss: 4.0479 - val_accuracy: 0.1300
Epoch 2/100
8757/8757 [==============================] - ETA: 35s - 

KeyboardInterrupt: 

In [ ]:
plt.plot(sim_conv_model.history.history['val_accuracy'])
plt.plot(sim_conv_model.history.history['accuracy'])
plt.title('val_accuracy')
plt.show()

plt.plot(sim_conv_model.history.history['val_loss'])
plt.plot(sim_conv_model.history.history['loss'])
plt.title('val_loss')
plt.show()